In [9]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

columns_to_use = ['KM_INT', 'FUEL','DRIVE','GEARBOX', 'AGE','Ford','Hyundai','Kia','Mahindra','Maruti Suzuki','Nissan','Tata','Toyota','Volkswagen','Black','Gray','Blue','Brown','Maroon','Red','Silver','Other Colors','Hatchback','Pickup','SUV','Sedan','1001-1500(cc)','1501-2000(cc)','2001-2500(cc)','2501-2900(cc)','2901-above(cc)']


df = pd.read_csv('Final-with-full-dummies_with_value_0and_1.csv');

In [10]:
print(df)

      Unnamed: 0  PRICE_NEW  KM_INT  FUEL  GEARBOX  DRIVE  AGE  Ford  Hyundai  \
0              0    1200000   96000     1        1      0   57     0        0   
1              1    1992000   75000     1        1      0   55     0        0   
2              2    1800000   70000     1        1      0   53     0        0   
3              3    6500000   78000     1        1      0   44     0        0   
4              4    2500000   98000     1        1      0   42     0        0   
5              5    2700000   46000     1        1      0   13     0        0   
6              9    3200000   40000     1        1      0   13     0        0   
7             10    7100000   88090     0        1      1   13     0        0   
8             11    3550000   85000     1        0      1   13     0        0   
9             12    3100000   53000     1        1      1   13     0        0   
10            13    3500000   75000     1        0      1   12     0        0   
11            14    1550000 

In [11]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan

pd.set_option('display.max_rows', None)

data=df.copy()
dependent_variable = 'PRICE_NEW'

data['AGE'] = data['AGE'].replace(0, 0.5);
df.head()
y = data[dependent_variable]
X = data[columns_to_use]


# print(model_linear.bse)


print('***********************Semi Log Summary*************************')
y_semi_log = np.log(y)
X_semi_log=X.copy()
X_semi_log = sm.add_constant(X_semi_log)  # Add a constant term for the intercept

model_semi_log = sm.OLS(y_semi_log, X_semi_log).fit()
print(model_semi_log.summary())
# print(model_semi_log.bse)
residuals = model_semi_log.resid

# Perform Breusch-Pagan test
# lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(residuals, X_semi_log)

# Print the results
# print("Breusch-Pagan test: Semi log")
# print("LM Statistic:", lm)
# print("LM Test p-value:", lm_p_value)
# print("F-Statistic:", fvalue)
# print("F-Test p-value:", f_p_value)







***********************Semi Log Summary*************************
                            OLS Regression Results                            
Dep. Variable:              PRICE_NEW   R-squared:                       0.686
Model:                            OLS   Adj. R-squared:                  0.682
Method:                 Least Squares   F-statistic:                     142.2
Date:                Sun, 07 Jul 2024   Prob (F-statistic):               0.00
Time:                        22:49:43   Log-Likelihood:                -834.02
No. Observations:                2045   AIC:                             1732.
Df Residuals:                    2013   BIC:                             1912.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

In [12]:
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data


# For Semi-Log Model
vif_semi_log = calculate_vif(X_semi_log)
# print('\nVIF for Semi-Log Model:')
# print(vif_semi_log)


In [20]:
print('Box COX Implementation')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from statsmodels.stats.diagnostic import het_breuschpagan
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment

# Assuming 'data' is your DataFrame
data = df.copy()

# Define the dependent and independent variables
dependent_variable = 'PRICE_NEW'

# Apply Box-Cox transformation to the dependent variable
y_transformed, lambda_value = stats.boxcox(data[dependent_variable])

print('*****lambda'+str(lambda_value)+'*****')
# Update the dependent variable in the DataFrame with the transformed values
data[dependent_variable] = y_transformed

# Separate the dependent and independent variables
y = data[dependent_variable]
X = data[columns_to_use]

# Add a constant term for the intercept
X_linear = sm.add_constant(X)

# Fit the linear regression model with the transformed dependent variable
print('***********************Box-Cox Transformed Summary*************************')
model_boxcox = sm.OLS(y, X_linear).fit()
print(model_boxcox.summary())



print('*************Export to excel the summary***************')

excel_file = 'linear_regression_summary.xlsx'
writer = pd.ExcelWriter(excel_file, engine='openpyxl')
summary = model_boxcox.summary2().tables
# Initialize a workbook and grab the active worksheet
workbook = Workbook()
worksheet = workbook.active
worksheet.title = 'Summary'

# Define a starting row
start_row = 1

# Function to write DataFrame to Excel starting from a specific row
def write_table_to_excel(table, start_row):
    df = pd.DataFrame(table)
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), start=start_row):
        for c_idx, value in enumerate(row, start=1):
            worksheet.cell(row=r_idx, column=c_idx, value=value)
            worksheet.cell(row=r_idx, column=c_idx).alignment = Alignment(horizontal='center', vertical='center')

# Write each table in the summary to Excel
for table in summary:
    write_table_to_excel(table, start_row)
    start_row += len(table) + 2  # Add some space between tables

# Save the Excel file
workbook.save('linear_regression_summary.xlsx')
print('*************End of Export to excel the summary***************')

predicted_boxcox = model_boxcox.predict(X_linear)

# Step 2: Transform predicted values back to original scale
predicted_original_scale = (np.power((predicted_boxcox * lambda_value) + 1, 1 / lambda_value))

# Step 3: Calculate residuals in original scale
residuals = data[dependent_variable] - predicted_original_scale

lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(residuals, X_linear)

print("Breusch-Pagan test: Box Cox Transformation ")
print("LM Statistic:", lm)
print("LM Test p-value:", lm_p_value)
print("F-Statistic:", fvalue)
print("F-Test p-value:", f_p_value)

# Step 4: Calculate RMSE
rmse_boxcox = np.sqrt(np.mean(residuals**2))

print("RMSE for Box-Cox transformed model:", rmse_boxcox)

Box COX Implementation
*****lambda-0.08319755830379594*****
***********************Box-Cox Transformed Summary*************************
                            OLS Regression Results                            
Dep. Variable:              PRICE_NEW   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     143.2
Date:                Mon, 08 Jul 2024   Prob (F-statistic):               0.00
Time:                        07:43:05   Log-Likelihood:                 1638.2
No. Observations:                2045   AIC:                            -3212.
Df Residuals:                    2013   BIC:                            -3032.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025    

In [17]:

# WLS model for removing heteroskedasticity
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan

pd.set_option('display.max_rows', None)

data = df.copy()
dependent_variable = 'PRICE_NEW'

data['AGE'] = data['AGE'].replace(0, 0.5);
df.head()

y = data[dependent_variable]
X = data[columns_to_use]


# Function to calculate weights based on squared residuals
def get_weights(resid):
    return 1 / np.square(resid)

# Run Semi-Log model (already in your code)
y_semi_log = np.log(y)
X_semi_log = sm.add_constant(X.copy())  # Add a constant term for the intercept
model_semi_log = sm.OLS(y_semi_log, X_semi_log).fit()
residuals = model_semi_log.resid

# Calculate weights based on squared residuals from Semi-Log model
weights = get_weights(residuals)

# Run WLS model with calculated weights
model_wls = sm.WLS(y_semi_log, X_semi_log, weights=weights).fit()
print('***********************WLS Summary*************************')
# print(model_wls.summary())

# Perform Breusch-Pagan test on WLS residuals
lm_wls, lm_wls_p_value, fvalue_wls, f_p_value_wls = het_breuschpagan(model_wls.resid, X_semi_log)

# Print WLS Breusch-Pagan test results
print("\nBreusch-Pagan test: WLS")
print("LM Statistic:", lm_wls)
print("LM Test p-value:", lm_wls_p_value)
print("F-Statistic:", fvalue_wls)
print("F-Test p-value:", f_p_value_wls)

***********************WLS Summary*************************

Breusch-Pagan test: WLS
LM Statistic: 393.3769883773763
LM Test p-value: 2.22485740878443e-64
F-Statistic: 15.466074827144336
F-Test p-value: 2.115254586309556e-72


In [ ]:
# Robust model  and WLS for removing heteroskedasticity

import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.robust.robust_linear_model import RLM

pd.set_option('display.max_rows', None)

data = df.copy()
dependent_variable = 'PRICE_NEW'

data['AGE'] = data['AGE'].replace(0, 0.5)  # Handle missing age values

y = data[dependent_variable]
X = data[columns_to_use]


# Function to calculate weights based on squared residuals
def get_weights(resid):
    return 1 / np.square(resid)


# Run Semi-Log model (already in your code)
y_semi_log = np.log(y)
X_semi_log = sm.add_constant(X.copy())  # Add a constant term for the intercept

# Perform Breusch-Pagan test on Semi-Log model
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(residuals, X_semi_log)




# Run WLS model with calculated weights
weights = get_weights(residuals)
model_wls = sm.WLS(y_semi_log, X_semi_log, weights=weights).fit()


# Perform Breusch-Pagan test on WLS residuals
lm_wls, lm_wls_p_value, fvalue_wls, f_p_value_wls = het_breuschpagan(model_wls.resid, X_semi_log)



# Run Robust Regression (MM-estimator)
model_robust = RLM(y_semi_log, X_semi_log, M=sm.robust.norms.HuberT()).fit()
print('\n***********************Robust Regression Summary*************************')
# print(model_robust.summary())

# Perform Breusch-Pagan test on Robust Regression residuals
lm_robust, lm_robust_p_value, fvalue_robust, f_p_value_robust = het_breuschpagan(model_robust.resid, X_semi_log)

# Print Robust Regression Breusch-Pagan test results
print("\nBreusch-Pagan test: Robust Regression")
print("LM Statistic:", lm_robust)
print("LM Test p-value:", lm_robust_p_value)
print("F-Statistic:", fvalue_robust)
print("F-Test p-value:", f_p_value_robust)


***********************Robust Regression Summary*************************

Breusch-Pagan test: Robust Regression
LM Statistic: 465.2841528075872
LM Test p-value: 6.092451211241076e-79
F-Statistic: 19.125877387222232
F-Test p-value: 8.311368406469039e-91


In [ ]:
#Log Log model
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan

pd.set_option('display.max_rows', None)

data=df.copy()
dependent_variable = 'PRICE_NEW'

data['AGE'] = data['AGE'].replace(0, 0.5);
df.head()
y = data[dependent_variable]
X = data[columns_to_use]

print('***********************Log Log Summary*************************')
X_log_log = X.copy()
X_log_log = sm.add_constant(X_log_log)  # Add a constant term for the intercept
y_log_log = np.log(y)

X_log_log[['KM_INT', 'AGE']] = np.log(X_log_log[['KM_INT', 'AGE']])  # Log-transform multiple independent variables
model_log_log = sm.OLS(y_log_log, X_log_log).fit()
# print(model_log_log.summary())
print('......Breisch-Pagan.......')

residuals = model_log_log.resid

# Perform Breusch-Pagan test
lml, lml_p_value, flvalue, fl_p_value = het_breuschpagan(residuals, X_log_log)


# Print the results
print("Breusch-Pagan test: Log log")
print("LM Statistic:", lml)
print("LM Test p-value:", lml_p_value)
print("F-Statistic:", flvalue)
print("F-Test p-value:", fl_p_value)

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data


# For Log-Log Model
vif_log_log = calculate_vif(X_log_log)
print('\nVIF for Log-Log Model:')
print(vif_log_log)

***********************Log Log Summary*************************
......Breisch-Pagan.......
Breusch-Pagan test: Log log
LM Statistic: 202.0085710434029
LM Test p-value: 5.482402362598229e-27
F-Statistic: 7.117517803223081
F-Test p-value: 1.2130437531284927e-28

VIF for Log-Log Model:
          Variable         VIF
0            const  445.501757
1           KM_INT    1.794765
2             FUEL    2.229955
3            DRIVE    1.780809
4          GEARBOX    1.130364
5              AGE    2.016460
6             Ford    3.949548
7          Hyundai    8.848615
8              Kia    3.171598
9         Mahindra    3.738604
10   Maruti Suzuki    7.969581
11          Nissan    2.534686
12            Tata    3.769562
13          Toyota    4.179719
14      Volkswagen    2.753006
15           Black    1.152069
16            Gray    1.677923
17            Blue    1.304027
18           Brown    1.174695
19          Maroon    1.190907
20             Red    1.338555
21          Silver    1.865506
22 

In [ ]:


# This is done to check the variables to be used as a predictor
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd


dfNewCopy = df.copy()
y = dfNewCopy['PRICE_NEW']

dfNewCopy.head()
# dfNewCopy = df.drop(columns=df.columns[0])

X=dfNewCopy.drop(['PRICE_NEW'],axis=1)
X = sm.add_constant(X)


# # print(X.info())
model = sm.OLS(y.astype(float), X.astype(float)).fit()
# print(model.fit())

# print(len(X))
# print(len(y))


# # Display the summary
# print(model.summary())

column_names = X.columns

# Extract the coefficients from the model
coefficients = model.params

# Create a string representation of the regression equation
equation = 'Y = {:.2f}'.format(coefficients['const'])

for i, coefficient in enumerate(coefficients[1:], start=1):
    equation += ' + {:.2f} * {}'.format(coefficient, column_names[i-1])

print('******************Equation of linear model of hedonic price is')
# Display the equation
print(equation)

# # Extracting p-values from the summary
# p_values = summary.tables[1]['P>|t|']
# print("P-values:")
# print(p_values)


******************Equation of linear model of hedonic price is
Y = 2509111.38 + 779.38 * const + -6.02 * Unnamed: 0 + -627379.27 * KM_INT + -295880.05 * FUEL + 491180.67 * GEARBOX + -134570.54 * DRIVE + -1523187.06 * AGE + -539424.68 * Ford + 364253.39 * Hyundai + -1480027.05 * Kia + 204462.77 * Mahindra + 644933.00 * Maruti Suzuki + 25092.99 * Nissan + -370465.87 * Renault + 3408660.07 * Tata + 1774813.81 * Toyota + 33259.74 * Volkswagen + 346657.97 * Black + 146034.86 * Blue + 308529.77 * Brown + 707719.60 * Gray + 151801.86 * Maroon + 163730.77 * Other Colors + 336246.57 * Red + 315130.24 * Silver + 894582.26 * White + -552141.19 * Hatchback + 1385667.45 * Pickup + 599333.27 * SUV + 181669.59 * Sedan + -913017.00 * Van + -769844.11 * 0-1000(cc) + -245297.80 * 1001-1500(cc) + 350656.36 * 1501-2000(cc) + 1497247.77 * 2001-2500(cc) + 2589366.15 * 2501-2900(cc)
